In [ ]:
pip install sentencepiece transformers

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from ast import literal_eval
import pickle
import time
from torch.utils.data import DataLoader
import torch.optim as optim

from transformers import T5ForConditionalGeneration, T5Tokenizer

summarizer_model = T5ForConditionalGeneration.from_pretrained("t5-large")
tokenizer = T5Tokenizer.from_pretrained("t5-large")

summarizer_model.eval()
for param in summarizer_model.parameters():
    param.requires_grad = False


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
news_df=pd.read_csv("/teamspace/studios/this_studio/Data/news_min (2).tsv", sep='\t')
news_df

,News ID,Category,Topic,Headline,News body
0,N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI..."
1,N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...
2,N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...
3,N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...
4,N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec..."
...,...,...,...,...,...
113757,N123757,sports,soccer_fifa_wwc,Hope who? Alyssa Naeher's penalty save sends U...,"LYON, France At the conclusion of the United..."
113758,N123758,sports,baseball_mlb,Chris Sale Explains What Specifically Has Gone...,The first half of Chris Sale's season could be...
113759,N123759,sports,basketball_nba_videos,Raptor fans jam streets to celebrate 1st NBA t...,Canadians are celebrating the country's first ...
113760,N123760,news,newspolitics,Judge won't allow Flynn to fire his attorneys,A federal judge denied the request by Michael ...


In [4]:
nid2body = dict(zip(news_df["News ID"], news_df["Headline"]))

# Example check
print(f"Total items: {len(nid2body)}")
sample_nid = list(nid2body.keys())[0]
print(f"Example:\nNID: {sample_nid}\nBody: {nid2body[sample_nid][:300]}")

Total items: 113762
Example:
NID: N10000
Body: Predicting Atlanta United's lineup against Columbus Crew in the U.S. Open Cup


In [5]:
# Path to your sid2text pickle file
with open('/teamspace/studios/this_studio/Data/sid2text.pkl', 'rb') as f:
    sid2text = pickle.load(f)
len(sid2text)

135001

In [6]:
# === Device and Precision Setup ===
torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 192

# === Utility Functions ===
def KDE_PMI(emb1, emb2, bandwidth=0.7):
    bandwidth = torch.tensor(bandwidth, dtype=torch.float32, device=emb1.device)
    diff = emb1 - emb2
    sim = torch.exp(- (diff ** 2) / (2 * bandwidth ** 2))
    return sim

def get_embedding(key, table, dim):
    if key not in table:
        table[key] = torch.nn.Parameter(torch.randn(dim, dtype=torch.float32, device=device) * 0.01, requires_grad=True)
    return table[key]


# === Load Embeddings ===
with open("/teamspace/studios/this_studio/Data/summary_embeddings_pca192.pkl", "rb") as f:
    summary_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}
with open("/teamspace/studios/this_studio/Data/newsbody_embeddings_pca192.pkl", "rb") as f:
    newsbody_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}
with open("/teamspace/studios/this_studio/Data/headline_embeddings_pca192.pkl", "rb") as f:
    headline_embed = {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in pickle.load(f).items()}

embed_tables = {
    'summary': summary_embed,
    'newsbody': newsbody_embed,
    'headline': headline_embed
}

# === Load Dataset ===
lookup_df = pd.read_csv("/teamspace/studios/this_studio/Data/Behavior_Vocab.csv").set_index('EdgeID')
train_df = pd.read_csv("/teamspace/studios/this_studio/Data/trainfile.csv")

In [7]:
tail_set = set()

print("Building Tail2Idx from Bhist and Bpos...")
for row in tqdm(train_df.itertuples(), total=len(train_df), desc="Collecting tails"):
    try:
        bhist = literal_eval(row.Bhist)
        bpos = row.Bpos

        # Add tails from Bhist
        for b_id in bhist:
            if b_id in lookup_df.index:
                tail = lookup_df.loc[b_id, 'Tail']
                tail_set.add(tail)

        # Add tail from Bpos (in case it's not already in Bhist)
        if bpos in lookup_df.index:
            tail = lookup_df.loc[bpos, 'Tail']
            tail_set.add(tail)

    except Exception as e:
        print(f"[Skip] Error in row {row.Index}: {e}")

# === Final mappings ===
tail_ids = sorted(tail_set)
tail2idx = {tid: idx for idx, tid in enumerate(tail_ids)}
idx2tail = {idx: tid for tid, idx in tail2idx.items()}

print(f"Tail2Idx built: {len(tail2idx)} unique tail IDs.")

Building Tail2Idx from Bhist and Bpos...


Tail2Idx built: 173540 unique tail IDs.


In [8]:
class BehaviorEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.W_dwell = nn.Linear(1, hidden_dim, bias=False)
        self.gamma_rel = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.y_e = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.f = nn.Parameter(torch.empty(1, dtype=torch.float32, device=device))
        self.y_s = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.y_focus = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.y_acc = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.K_short = nn.Parameter(torch.randn(30, dtype=torch.float32, device=device))  # last 50 steps
        self.K_long  = nn.Parameter(torch.randn(10, dtype=torch.float32, device=device))
        self.K_event = nn.Parameter(torch.randn(30, dtype=torch.float32, device=device))


        self.classifier = nn.Linear(hidden_dim, len(tail2idx))  # for tail_id classification
        self.bpos_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        self.init_parameters()

    def init_parameters(self):
        nn.init.kaiming_uniform_(self.W_dwell.weight, nonlinearity='linear')
        for param in [self.gamma_rel, self.y_e, self.y_s, self.y_focus, self.y_acc]:
            nn.init.normal_(param, mean=0.0, std=0.01)
        with torch.no_grad():
            self.f.copy_(torch.tensor([0.5], dtype=torch.float32, device=device))

    def forward(self, Bhist, Bpos, lookup_df, tail2idx, embed_tables):
        total_loss = torch.tensor(0., dtype=torch.float32, device=device)
        enc_loss = torch.tensor(0., dtype=torch.float32, device=device)
        pred_loss = torch.tensor(0., dtype=torch.float32, device=device)

        prev_e_b = torch.zeros(hidden_dim, dtype=torch.float32, device=device)
        memory = []

        for t, b_id in enumerate(Bhist):
            if b_id not in lookup_df.index:
                continue

            row = lookup_df.loc[b_id]
            head_id, rel, tail_id = row['Head'], row['Relation'], row['Tail']
            dwell_s = row['dwell'] / 130000.0 if pd.notna(row['dwell']) else None

            head_emb = prev_e_b if t > 0 else embed_tables['headline'].get(head_id, torch.zeros(hidden_dim, device=device))
            e_b = torch.zeros(hidden_dim, dtype=torch.float32, device=device)
            mem_e_b = torch.zeros(hidden_dim, dtype=torch.float32, device=device)
            

            if rel in ['click', 'skip', 'gen_summ']:
                tail_emb = embed_tables['newsbody'].get(tail_id, torch.zeros(hidden_dim, device=device))
                if rel == 'click':
                    dwell_v = self.W_dwell(torch.tensor([[dwell_s]], dtype=torch.float32, device=device)).squeeze(0) if dwell_s else torch.zeros(hidden_dim, device=device)
                    e_b = tail_emb + (dwell_v * self.y_e * KDE_PMI(tail_emb, head_emb))
                elif rel == 'skip':
                    e_b = self.f * tail_emb + (1. - self.f) * KDE_PMI(tail_emb, head_emb)
                elif rel == 'gen_summ':
                    hl_emb = embed_tables['headline'].get(tail_id, torch.zeros(hidden_dim, device=device))
                    e_b = self.y_s * KDE_PMI(tail_emb, head_emb) * hl_emb

            elif rel == 'summ_gen':
                sum_emb = embed_tables['summary'].get(tail_id, torch.zeros(hidden_dim, device=device))
                doc_emb = embed_tables['newsbody'].get(head_id, torch.zeros(hidden_dim, device=device))
                e_b = self.y_focus * KDE_PMI(sum_emb, head_emb) + self.y_acc * KDE_PMI(sum_emb, doc_emb)
            else:
                tail_emb = embed_tables['newsbody'].get(tail_id, torch.zeros(hidden_dim, device=device))
                e_b = tail_emb

            # Apply γ to e_b
            e_b = self.gamma_rel * e_b
            e_b_orig = e_b.clone()
            # --- MEMORY INJECTION ---
            if memory:
                idxs = torch.arange(len(memory)-1, -1, -1, device=device)
                memory_stack = torch.stack(memory)

                w_short = self.K_short[idxs.clamp(max=len(self.K_short)-1)]
                w_long  = self.K_long [idxs.clamp(max=len(self.K_long )-1)]
                w_event = self.K_event[idxs.clamp(max=len(self.K_event)-1)]

                m_short = (w_short.unsqueeze(1) * memory_stack).sum(0)
                m_long  = (w_long .unsqueeze(1) * memory_stack).sum(0)
                m_event = (w_event.unsqueeze(1) * memory_stack).sum(0)

                gates = torch.softmax(torch.stack([m_short, m_long, m_event]), dim=0)
                mem_e_b = gates[0]*m_short + gates[1]*m_long + gates[2]*m_event

            memory.append(mem_e_b.detach())  # Detach to stop gradient
            e_b = e_b_orig + mem_e_b
            prev_e_b = e_b

            # BCE or CE loss over current tail_id
            if tail_id in tail2idx:
                logits = self.classifier(e_b.unsqueeze(0))
                target = torch.tensor([tail2idx[tail_id]], dtype=torch.long, device=device)
                enc_loss += F.cross_entropy(logits, target)

        # === Final Step Prediction ===
        e_b_pos_pred = self.bpos_mlp(e_b)
        b_next = lookup_df.loc[Bpos]
        tpos_id = b_next['Tail']
        logits_pos = self.classifier(e_b_pos_pred.unsqueeze(0))
        target_pos = torch.tensor([tail2idx[tpos_id]], dtype=torch.long, device=device)
        pred_loss = F.cross_entropy(logits_pos, target_pos)

        total_loss = (0.5 * enc_loss + 0.5 * pred_loss)/(100*len(Bhist)+1)
        return e_b, e_b_pos_pred, total_loss


In [9]:

class SummaryInverseMapper(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, e_b_pos, e_b_hist, e_doc):
        x = torch.cat([e_b_pos, e_b_hist, e_doc], dim=-1)
        return self.fc(x)


class CrossAttentionAdapter(nn.Module):
    def __init__(self, query_dim, key_value_dim, hidden_dim):
        super().__init__()
        self.query_proj = nn.Linear(query_dim, hidden_dim)
        self.key_proj = nn.Linear(key_value_dim, hidden_dim)
        self.value_proj = nn.Linear(key_value_dim, hidden_dim)
        self.out_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, query, key, value):
        if query.dim() == 1: query = query.unsqueeze(0)
        if key.dim() == 1: key = key.unsqueeze(0)
        if value.dim() == 1: value = value.unsqueeze(0)

        Q = self.query_proj(query).unsqueeze(1)  # [B, 1, H]
        K = self.key_proj(key).unsqueeze(1)      # [B, 1, H]
        V = self.value_proj(value).unsqueeze(1)  # [B, 1, H]

        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / (Q.size(-1) ** 0.5)
        attn_weights = F.softmax(attn_scores, dim=-1)
        attended = torch.matmul(attn_weights, V)
        return self.out_proj(attended.squeeze(1))


class B2SModel(nn.Module):
    def __init__(self, encoder, inverse_mapper, adapter, summarizer_model, tokenizer, nid2body, sid2text):
        super().__init__()
        self.encoder = encoder
        self.inverse_mapper = inverse_mapper
        self.adapter = adapter
        self.summarizer = summarizer_model  # T5ForConditionalGeneration
        self.tokenizer = tokenizer
        self.nid2body = nid2body
        self.sid2text = sid2text
        self.adapter_to_t5 = nn.Linear(adapter.out_proj.out_features, summarizer_model.config.d_model)

    def forward(self, Bhist, Bpos, lookup_df, tail2idx, embed_tables):
        device = next(self.parameters()).device

        # Step 1: Encode user behavior
        e_b, e_b_pos_pred, loss_enc = self.encoder(Bhist, Bpos, lookup_df, tail2idx, embed_tables)

        # Step 2: Get document embedding and text
        head_id = lookup_df.loc[Bpos]['Head']
        tail_id = lookup_df.loc[Bpos]['Tail']
        e_doc = embed_tables['newsbody'].get(head_id, torch.zeros_like(e_b))
        doc_text = self.nid2body.get(head_id, "")

        # Step 3: Predict personalized summary embedding
        e_s_pos_pred = self.inverse_mapper(e_b_pos_pred, e_b, e_doc)
        adapted = self.adapter(e_s_pos_pred, e_doc, e_b)
        adapted_proj = self.adapter_to_t5(adapted)  # [1, d_model]

        # Step 4: Encode document text with T5
        input_enc = self.tokenizer(doc_text, return_tensors="pt", padding=True, truncation=True).to(device)
        encoder_outputs = self.summarizer.encoder(
            input_ids=input_enc['input_ids'],
            attention_mask=input_enc['attention_mask'],
            return_dict=True
        )

        # Step 5: Prepare teacher-forced decoder inputs
        sid = lookup_df.loc[Bpos]['Tail']
        gold_summary = self.sid2text.get(sid, "")
        gold_inputs = self.tokenizer(gold_summary, return_tensors="pt", padding=True, truncation=True).to(device)
        decoder_input_ids = gold_inputs['input_ids'][:, :-1].contiguous()
        decoder_labels = gold_inputs['input_ids'][:, 1:].contiguous()

        # Step 6: Inject adapter vector as first token
        adapter_token_embed = adapted_proj.unsqueeze(1)  # [1, 1, d_model]
        dec_embed = self.summarizer.get_input_embeddings()(decoder_input_ids)  # [1, T, d_model]
        final_embed = torch.cat([adapter_token_embed, dec_embed], dim=1)

        decoder_labels = torch.cat([
            torch.full((1, 1), -100, dtype=torch.long, device=device),  # mask adapter loss
            decoder_labels
        ], dim=1)

        # Step 7: Decode with adapter-injected embeddings
        outputs = self.summarizer(
            encoder_outputs=encoder_outputs,
            decoder_inputs_embeds=final_embed,
            labels=decoder_labels
        )
        loss_txt = outputs.loss

        # Step 8: Decode predicted summary
        pred_ids = torch.argmax(outputs.logits, dim=-1)
        pred_summary = self.tokenizer.decode(pred_ids[0], skip_special_tokens=True)

        # Step 9: Return losses and summaries
        impersum_loss = 0.5 * loss_enc + 0.5 * loss_txt
        return (impersum_loss, loss_txt, loss_enc), pred_summary, gold_summary, doc_text


In [10]:
# === Define Supporting Modules ===
inverse_mapper = SummaryInverseMapper(hidden_dim).to(device)
adapter = CrossAttentionAdapter(query_dim=192, key_value_dim=192, hidden_dim=192).to(device)

encoder = BehaviorEncoder().to(device)

# === Initialize the Combined B2S Model ===
b2s_model = B2SModel(
    encoder=encoder,
    inverse_mapper=inverse_mapper,
    adapter=adapter,
    summarizer_model=summarizer_model,
    tokenizer=tokenizer,
    nid2body=nid2body,
    sid2text=sid2text
).to(device)


In [11]:
train_df_subset = train_df.iloc[:550].reset_index(drop=True)
train_rows = train_df_subset.iloc[:500]
test_rows = train_df_subset.iloc[500:550]



def extract_pairs(df, lookup_df, tail2idx):
    data_pairs = []
    for _, row in df.iterrows():
        try:
            Bhist = literal_eval(row['Bhist'])
            Bpos = row['Bpos']  # explicitly given next behavior
            if Bpos in lookup_df.index:
                tail_id = lookup_df.loc[Bpos, 'Tail']
                if tail_id in tail2idx:
                    data_pairs.append((Bhist, Bpos))  # use full Bhist, and correct Bpos
        except:
            continue
    return data_pairs


train_data = extract_pairs(train_rows, lookup_df, tail2idx)
test_data = extract_pairs(test_rows, lookup_df, tail2idx)

print(f"Train samples: {len(train_data)} | Test samples: {len(test_data)}")

Train samples: 500 | Test samples: 50


In [13]:
# === Pick one sample ===
sample_idx = random.randint(0, len(train_data) - 1)
Bhist, Bpos = train_data[sample_idx]

print(f"\n=== Running B2S Forward Pass ===")
print(f"Sample Index: {sample_idx}")
print(f"Behavior History Length: {len(Bhist)}")
print(f"Bpos: {Bpos}")

b2s_model.eval()
with torch.no_grad():
    e_b, e_b_pos_pred, _ = b2s_model.encoder(Bhist, Bpos, lookup_df, tail2idx, embed_tables)

    head_id = lookup_df.loc[Bpos]['Head']
    e_doc = embed_tables['newsbody'].get(head_id, torch.zeros_like(e_b))

    e_s_pos_pred = b2s_model.inverse_mapper(e_b_pos_pred, e_b, e_doc)

    # Run model to get detailed losses
    (loss_total, loss_gen, loss_enc), pred_summary, gold_summary, generic_summary = b2s_model(
        Bhist, Bpos, lookup_df, tail2idx, embed_tables
    )

# === Metadata ===
true_tail_id = lookup_df.loc[Bpos]['Tail']
print(f"\nPredicted e_b_pos shape: {e_b_pos_pred.shape}")
print(f"True Tail ID (Bpos): {true_tail_id}")

# === Summaries ===
print("\n--- Personalized Summary Output ---")
print(f"Generic Summary   : {generic_summary}")
print(f"Gold Summary   : {gold_summary}")
print(f"Predicted      : {pred_summary}")

# === Losses ===
print("\n--- Loss Breakdown ---")
print(f"Total Loss     : {loss_total:.4f}")
print(f"Gen Loss       : {loss_gen:.4f}")
print(f"Encoder Loss   : {loss_enc:.4f}")


=== Running B2S Forward Pass ===
Sample Index: 127
Behavior History Length: 133
Bpos: B5327



Predicted e_b_pos shape: torch.Size([192])
True Tail ID (Bpos): S-128

--- Personalized Summary Output ---
Generic Summary   : AGUAS VERDES, Peru (AP)   Tired and thirsty, Betania Ramírez crossed into Peru on Friday with her 1-year-old girl in her arms and her 8-year-old boy beside her.
Gold Summary   : AGUAS VERDES, Peru (AP)   Tired and thirsty, Betania Ramírez crossed into Peru on Friday with her 1-year-old girl in her arms and her 8-year-old boy beside her.
Predicted      : ))  ERS    )) ,,,)) however ,,s,,,,,,,, und----,-, --year-- ,

--- Loss Breakdown ---
Total Loss     : 5.0444
Gen Loss       : 10.0281
Encoder Loss   : 0.0608


In [ ]:
import torch
from tqdm import tqdm
import os

# === Config ===
lr = 1e-3
num_epochs = 100
save_every = 10
checkpoint_template = "b2s_model_epoch{:03d}.pt"
resume = True  # set to False if starting fresh
start_epoch = 0

# === Optimizer ===
optimizer = torch.optim.Adam(b2s_model.parameters(), lr=lr)

# === Load Checkpoint if Resume Enabled ===
if resume:
    checkpoint_files = [f for f in os.listdir() if f.startswith("b2s_model_epoch") and f.endswith(".pt")]
    if checkpoint_files:
        last_ckpt = sorted(checkpoint_files)[-1]
        ckpt_epoch = int(last_ckpt.split("epoch")[-1].split(".pt")[0])
        start_epoch = ckpt_epoch
        b2s_model.load_state_dict(torch.load(last_ckpt))
        print(f"🔁 Loaded checkpoint '{last_ckpt}' and resumed from epoch {start_epoch + 1}")

# === Training Loop ===
for epoch in range(start_epoch, num_epochs):
    b2s_model.train()
    total_loss = 0.0
    total_gen_loss = 0.0

    print(f"\n🟦 Epoch {epoch + 1}/{num_epochs}")
    pbar = tqdm(train_data, desc=f"Training Epoch {epoch + 1}", dynamic_ncols=True)

    for Bhist, Bpos in pbar:
        optimizer.zero_grad()

        try:
            (loss_total, loss_txt, loss_enc), pred_summary, gold_summary, generic_summary = b2s_model(
                Bhist, Bpos, lookup_df, tail2idx, embed_tables
            )
        except Exception as e:
            pbar.set_postfix_str(f"[Skip {Bpos}] {str(e)}")
            continue

        loss_total.backward()
        optimizer.step()

        total_loss += loss_total.item()
        total_gen_loss += loss_txt.item()

        pbar.set_postfix({
            "Loss": f"{loss_total.item():.4f}",
            "GenLoss": f"{loss_txt.item():.4f}"
        })

    avg_loss = total_loss / len(train_data)
    avg_gen_loss = total_gen_loss / len(train_data)
    print(f"✅ [Epoch {epoch + 1}] Avg Loss: {avg_loss:.4f} | Gen Loss: {avg_gen_loss:.4f}")

    if (epoch + 1) % save_every == 0:
        ckpt_path = checkpoint_template.format(epoch + 1)
        torch.save(b2s_model.state_dict(), ckpt_path)
        print(f"📌 Saved checkpoint: {ckpt_path}")



🟦 Epoch 1/100


Training Epoch 1:  33%|███▎      | 165/500 [00:43<01:08,  4.92it/s, Loss=3.0953, GenLoss=6.1268] 

In [ ]:
# === Re-initialize your model architecture (same as training) ===
encoder = BehaviorEncoder().to(device)
inverse_mapper = SummaryInverseMapper(hidden_dim).to(device)
adapter = CrossAttentionAdapter(query_dim=192, key_value_dim=192, hidden_dim=192).to(device)

b2s_model = B2SModel(
    encoder=encoder,
    inverse_mapper=inverse_mapper,
    adapter=adapter,
    summarizer_model=summarizer_model,
    tokenizer=tokenizer,
    nid2body=nid2body,
    sid2text=sid2text
).to(device)

# === Load weights ===
checkpoint_path = "b2s_model_epoch010.pt"  # <-- change as needed
b2s_model.load_state_dict(torch.load(checkpoint_path, map_location=device))
print(f"✅ Loaded model checkpoint: {checkpoint_path}")


In [ ]:
import pandas as pd
from tqdm import tqdm

b2s_model.eval()
results = []

with torch.no_grad():
    for Bhist, Bpos in tqdm(test_data, desc="🔍 Evaluating"):
        try:
            loss, pred_summary, gold_summary, generic_summary = b2s_model(Bhist, Bpos, lookup_df, tail2idx, embed_tables)
            true_tail = lookup_df.loc[Bpos, 'Tail']
        except Exception as e:
            print(f"[Skip] {Bpos}: {e}")
            continue

        results.append({
            'Bpos': Bpos,
            'true_tail_id': true_tail,
            'generic summary':generic_summary,
            'pred_summary': pred_summary,
            'gold_summary': gold_summary
        })

df = pd.DataFrame(results)
# df.to_csv("b2s_eval_results.csv", index=False)
print("✅ Results saved to: b2s_eval_results.csv")